In [1]:
import sqlite3
import pandas as pd
import folium
import math

In [2]:
conn = sqlite3.connect('BBDD/aguaCHJucar.db')

cursor = conn.cursor()

In [35]:
df_loc

,latitude_copernicus,longitude_copernicus,latitude_embalses,longitude_embalses,latitude_aemet,longitude_aemet,latitude_rios_canales,longitude_rios_canales,nombre_aemet,estacion_aforo_rios_canales,embalse
0,39.174,-1.755,None,None,38.954167,-1.856389,39.137609,-1.749414,ALBACETE BASE AÉREA,LOS FRAILES,No hay embalse
1,39.174,-1.755,None,None,38.935833,-1.660000,39.137609,-1.749414,CHINCHILLA,LOS FRAILES,No hay embalse
2,39.174,-1.755,None,None,39.005833,-1.862778,39.137609,-1.749414,ALBACETE,LOS FRAILES,No hay embalse
3,39.174,-1.755,None,None,38.954167,-1.856389,39.186302,-1.740179,ALBACETE BASE AÉREA,MAHORA,No hay embalse
4,39.174,-1.755,None,None,38.935833,-1.660000,39.186302,-1.740179,CHINCHILLA,MAHORA,No hay embalse
5,39.174,-1.755,None,None,39.005833,-1.862778,39.186302,-1.740179,ALBACETE,MAHORA,No hay embalse


iodsiasdfi


In [45]:
pixel = 189

query = f'''
    SELECT 
        loc_copernicus.latitude AS latitude_copernicus,
        loc_copernicus.longitude AS longitude_copernicus,
        loc_embalses.latitude AS latitude_embalses,
        loc_embalses.longitude AS longitude_embalses,
        loc_aemet.latitude AS latitude_aemet,
        loc_aemet.longitude AS longitude_aemet,
        loc_rios_canales.latitude AS latitude_rios_canales,
        loc_rios_canales.longitude AS longitude_rios_canales,
        -- Información adicional
        aemet_info.nombre AS nombre_aemet,
        rios_canales_info.EstacióndeAforo AS estacion_aforo_rios_canales,
        -- Usar COALESCE para manejar los valores NULL
        COALESCE(embalses_info.Embalse, 'No hay embalse') AS embalse
    FROM df_pixeles_cercanos p
    -- Relacionar las localizaciones de la tabla df_pixeles_cercanos con las latitudes y longitudes
    INNER JOIN locations_id loc_copernicus ON loc_copernicus.location_id = p.location_id_copernicus
    LEFT JOIN locations_id loc_embalses ON loc_embalses.location_id = p.location_id_embalses
    INNER JOIN locations_id loc_aemet ON loc_aemet.location_id = p.location_id_aemet
    INNER JOIN locations_id loc_rios_canales ON loc_rios_canales.location_id = p.location_id_rios_canales
    -- Relacionar con tablas de información adicional
    INNER JOIN df_aemet_info aemet_info ON aemet_info.location_id = loc_aemet.location_id
    INNER JOIN df_rios_canales_info rios_canales_info ON rios_canales_info.location_id = loc_rios_canales.location_id
    LEFT JOIN df_embalses_info embalses_info ON embalses_info.location_id = loc_embalses.location_id
    -- Filtrar por location_id_copernicus = {pixel}
    WHERE p.location_id_copernicus = {pixel};
'''


# Ejecutar la consulta
cursor.execute(query)


df_loc = pd.read_sql_query(query, conn)

# Coordenadas base para centrar el mapa
map_center = [39.0, -0.5]  # Aproximadamente en la cuenca del Júcar

# Crear el mapa base
m = folium.Map(location=map_center, zoom_start=9.5)

# Añadir el cuadrado de la zona de Copernicus (representando un área más grande)
for index, row in df_loc.iterrows():
    # Longitud y latitud del centro de Copernicus
    lat_copernicus = row['latitude_copernicus']
    lon_copernicus = row['longitude_copernicus']

    # Definir un lado del cuadrado basado en 0.5 grados
    size_lat = 0.5  # 0.5 grados de latitud
    size_lon = 0.5 * 111.32 * math.cos(math.radians(lat_copernicus)) / 111.32  # Ajustado por la latitud (en grados)

    additional_lon = 5 / (111.32 * math.cos(math.radians(lat_copernicus))) 
    # Crear las coordenadas de las esquinas del cuadrado
    bounds = [
        [lat_copernicus - size_lat / 2, lon_copernicus - size_lon / 2],  # Esquina inferior izquierda
        [lat_copernicus + size_lat / 2, lon_copernicus + size_lon / 2 + additional_lon]   # Esquina superior derecha
    ]
    
    # Añadir el cuadrado de Copernicus al mapa
    folium.Rectangle(
        bounds=bounds,
        color='blue',
        weight=2,
        fill=True,
        fill_color='blue',
        fill_opacity=0,
        popup='Copernicus Area'
    ).add_to(m)

    # Añadir marcador para Aemet
    folium.Marker(
        location=[row['latitude_aemet'], row['longitude_aemet']],
        popup=f"Aemet: {row['nombre_aemet']}",
        icon=folium.Icon(color='red')
    ).add_to(m)

    # Añadir marcador para Ríos Canales
    folium.Marker(
        location=[row['latitude_rios_canales'], row['longitude_rios_canales']],
        popup= f"Ríos Canales: {row['estacion_aforo_rios_canales']}",
        icon=folium.Icon(color='purple')
    ).add_to(m)
    # Añadir marcador para embalses
    if df_loc['embalse'][0] != 'No hay embalse':
        folium.Marker(
            location=[row['latitude_embalses'], row['longitude_embalses']],
            popup=f"Embalse: {row['embalse']}",
            icon=folium.Icon(color='orange')
        ).add_to(m)

# Mostrar el mapa
m

### Extracción de datos

In [46]:
query = f'''
        SELECT 
            d.date,
            c.total_precipitation,
            c.skin_temperature,
            c.evaporation,
            c.runoff,
            c.snowfall,
            c.soil_water_l1,
            c.soil_water_l2,
            c.soil_water_l3,
            c.soil_water_l4,
            c.high_vegetation_cover,
            c.low_vegetation_cover 
            FROM df_copernicus c JOIN df_date d ON d.date_id = c.date_id  WHERE c.location_id = {pixel};
        '''
# Ejecutar la consulta
cursor.execute(query)


df_c = pd.read_sql_query(query, conn)
query = f'''
        SELECT 
            d.date,
            aem.tmed,
            aem.prec,
            aem.location_id
            FROM df_aemet aem 
            JOIN df_date d ON d.date_id = aem.date_id 
            JOIN df_pixeles_cercanos pc ON pc.location_id_aemet = aem.location_id 
            WHERE pc.location_id_copernicus =  {pixel};
        '''
# Ejecutar la consulta
cursor.execute(query)


df_aemet = pd.read_sql_query(query, conn)
query = f'''
        SELECT 
            d.date,
            rios.quantity_hm3,
            rios.location_id
            FROM df_rios_canales rios 
            JOIN df_date d ON d.date_id = rios.date_id 
            JOIN df_pixeles_cercanos pc ON pc.location_id_rios_canales = rios.location_id 
            WHERE pc.location_id_copernicus = {pixel};
        '''
# Ejecutar la consulta
cursor.execute(query)


df_rios = pd.read_sql_query(query, conn)

query = f'''
        SELECT 
            d.date,
            emb.quantity_hm3,
            emb.location_id
            FROM df_embalses emb 
            JOIN df_date d ON d.date_id = emb.date_id 
            JOIN df_pixeles_cercanos pc ON pc.location_id_embalses = emb.location_id 
            WHERE pc.location_id_copernicus = {pixel};
        '''
# Ejecutar la consulta
cursor.execute(query)


df_embalses = pd.read_sql_query(query, conn)

### Análisis

In [14]:
df_c

,date,total_precipitation,skin_temperature,evaporation,runoff,snowfall,soil_water_l1,soil_water_l2,soil_water_l3,soil_water_l4,high_vegetation_cover,low_vegetation_cover
0,1960-01-01,0.000000e+00,280.41217,-0.000852,0.000000,0.0,2.278468,2.367908,2.146910,1.280457,0.451935,0.517842
1,1960-01-02,2.218061e-06,280.50934,-0.000578,0.000000,0.0,2.262390,2.354335,2.149596,1.280975,0.451935,0.517842
2,1960-01-03,4.461035e-07,280.80370,-0.000641,0.000000,0.0,2.245047,2.336738,2.148508,1.281540,0.451935,0.517842
3,1960-01-04,0.000000e+00,279.93958,-0.000702,0.000000,0.0,2.223495,2.315618,2.143630,1.282066,0.451935,0.517842
4,1960-01-05,8.365605e-07,279.42030,-0.000587,0.000000,0.0,2.210156,2.302502,2.141049,1.282600,0.451935,0.517842
...,...,...,...,...,...,...,...,...,...,...,...,...
2034,1965-07-27,2.082346e-04,295.74698,-0.001326,0.000002,0.0,1.231328,1.347859,1.668715,1.515251,0.451935,0.517842
2035,1965-07-28,1.179023e-04,296.96620,-0.001569,0.000001,0.0,1.205596,1.342871,1.663166,1.514969,0.451935,0.517842
2036,1965-07-29,0.000000e+00,299.23434,-0.001912,0.000000,0.0,1.167106,1.341242,1.662134,1.514519,0.451935,0.517842
2037,1965-07-30,0.000000e+00,300.99152,-0.001987,0.000000,0.0,1.130051,1.341449,1.661623,1.514008,0.451935,0.517842


In [13]:
df_rios.groupby('date').mean()['quantity_hm3']

date
1912-01-01    2.258712
1912-01-02    2.248733
1912-01-03    2.130430
1912-01-04    2.066256
1912-01-05    2.078590
                ...   
2024-09-05    1.661040
2024-09-06    1.609200
2024-09-07    1.325981
2024-09-08    1.065355
2024-09-09    1.055290
Name: quantity_hm3, Length: 37235, dtype: float64

In [10]:
df_embalses.groupby('date').mean()['quantity_hm3']

date
1970-02-01    11.04
1970-02-02    11.32
1970-02-03    10.78
1970-02-04    11.32
1970-02-05    11.32
              ...  
2021-09-26     2.67
2021-09-27     2.65
2021-09-28     2.63
2021-09-29     2.59
2021-09-30     2.55
Name: quantity_hm3, Length: 18465, dtype: float64

In [ ]:
conn = sqlite3.connect('BBDD/aguaCHJucar.db')
cursor = conn.cursor()

query = '''
SELECT 
    d.date AS fecha,
    c.total_precipitation,
    c.skin_temperature,
    c.evaporation,
    c.runoff,
    c.snowfall,
    c.soil_water_l1,
    c.soil_water_l2,
    c.soil_water_l3,
    c.soil_water_l4,
    c.high_vegetation_cover,
    c.low_vegetation_cover,
    e.quantity_hm3 AS quantity_hm3_embalses,
    a.tmed,
    a.prec,
    a.tmin,
    a.tmax,
    a.dir,
    a.velmedia,
    a.racha,
    r.quantity_hm3 AS quantity_hm3_rios_canales
FROM df_pixeles_cercanos p
-- Subconsulta para obtener los location_ids relacionados con location_id_copernicus = 14
INNER JOIN (
    SELECT location_id_embalses, location_id_aemet, location_id_rios_canales
    FROM df_pixeles_cercanos
    WHERE location_id_copernicus = 14
) rel ON p.location_id_embalses = rel.location_id_embalses
        AND p.location_id_aemet = rel.location_id_aemet
        AND p.location_id_rios_canales = rel.location_id_rios_canales
-- Unir las tablas con las series temporales correspondientes
LEFT JOIN df_embalses e ON e.location_id = p.location_id_embalses
LEFT JOIN df_aemet a ON a.location_id = p.location_id_aemet
LEFT JOIN df_rios_canales r ON r.location_id = p.location_id_rios_canales
LEFT JOIN df_copernicus c ON c.location_id = p.location_id_copernicus
LEFT JOIN df_date d ON d.date_id = c.date_id
WHERE p.location_id_copernicus = 14
ORDER BY d.date;
'''

# Ejecutar la consulta
cursor.execute(query)


df = pd.read_sql_query(query, conn)